In [2]:
!pip install webrtcvad
!pip install ffmpeg-python
!pip install torchaudio transformers

In [3]:
!ffprobe -i /content/Heretic.2024.SAMPLE.mkv

ffprobe version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2007-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --ena

In [4]:
import webrtcvad
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import ffmpeg
from datetime import timedelta


def extract_audio(video_path, audio_path):
    """
    Извлечение английской аудиодорожки из видео с перезаписью существующего файла.
    """
    try:
        # Указываем английскую аудиодорожку (Stream #0:4)
        ffmpeg.input(video_path).output(
            audio_path, format='wav', acodec='pcm_s16le', ac=1, ar='16000', map='0:1'
        ).run(overwrite_output=True, capture_stderr=True)
        print(f"Аудио успешно извлечено: {audio_path}")
    except ffmpeg.Error as e:
        stderr_output = e.stderr.decode() if e.stderr else "Нет вывода stderr."
        print(f"Ошибка при извлечении аудио: {stderr_output}")
        raise e




def apply_vad(audio_path, frame_duration_ms=30):
    """
    Применение Voice Activity Detection для определения сегментов речи.
    """
    vad = webrtcvad.Vad(3)
    waveform, sample_rate = torchaudio.load(audio_path)

    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    audio = waveform.squeeze().numpy()

    audio_pcm = (audio * 32768).astype("int16").tobytes()
    frame_size = int(16000 * frame_duration_ms / 1000) * 2
    segments = []
    start_time = None

    for i in range(0, len(audio_pcm), frame_size):
        frame = audio_pcm[i:i + frame_size]
        if len(frame) < frame_size:
            break

        is_speech = vad.is_speech(frame, 16000)
        if is_speech:
            if start_time is None:
                start_time = i / (16000 * 2)
        elif start_time is not None:
            end_time = i / (16000 * 2)
            segments.append((start_time, end_time))
            start_time = None

    if start_time is not None:
        segments.append((start_time, len(audio_pcm) / (16000 * 2)))

    return segments


def load_model(model_name="jonatasgrosman/wav2vec2-large-xlsr-53-russian"):
    """
    Загрузка более точной модели Wav2Vec2.
    """
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    model = Wav2Vec2ForCTC.from_pretrained(model_name)
    print(f"Модель {model_name} загружена.")
    return processor, model



def transcribe_segment(audio_path, segment, processor, model, max_duration=15):
    """
    Распознавание текста для заданного сегмента.
    """
    waveform, rate = torchaudio.load(audio_path)
    start_sample = int(segment[0] * rate)
    end_sample = int(segment[1] * rate)
    segment_waveform = waveform[:, start_sample:end_sample]

    segment_duration = segment_waveform.size(1) / rate
    if segment_duration > max_duration:
        print(f"Сегмент слишком длинный ({segment_duration:.2f} сек.), разделение...")
        mid_time = (segment[0] + segment[1]) / 2
        return (transcribe_segment(audio_path, (segment[0], mid_time), processor, model) +
                transcribe_segment(audio_path, (mid_time, segment[1]), processor, model))

    input_values = processor(segment_waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription


def generate_srt(segments, transcriptions, output_path):
    """
    Генерация файла субтитров в формате .srt.
    """
    srt_content = []
    for i, (segment, transcription) in enumerate(zip(segments, transcriptions)):
        start_time = str(timedelta(seconds=segment[0]))[:-3]
        end_time = str(timedelta(seconds=segment[1]))[:-3]
        srt_content.append(f"{i+1}\n{start_time} --> {end_time}\n{transcription}\n")

    with open(output_path, 'w', encoding='utf-8') as file:
        file.write("\n".join(srt_content))
    print(f"Субтитры сохранены: {output_path}")


def main(video_path, audio_path, srt_path):
    # Извлечение аудио
    extract_audio(video_path, audio_path)

    # Применение VAD для определения сегментов
    segments = apply_vad(audio_path)
    print(f"Обнаружено {len(segments)} сегментов речи.")

    # Загрузка модели Wav2Vec2
    processor, model = load_model()

    # Распознавание текста для каждого сегмента
    transcriptions = []
    for idx, segment in enumerate(segments):
        print(f"Распознавание сегмента {idx+1}/{len(segments)} ({(idx+1)/len(segments)*100:.2f}%)...")
        try:
            transcription = transcribe_segment(audio_path, segment, processor, model)
            transcriptions.append(transcription)
        except Exception as e:
            print(f"Ошибка в сегменте {idx+1}: {e}")
            transcriptions.append("[НЕ УДАЛОСЬ РАСПОЗНАТЬ]")

    # Генерация субтитров
    generate_srt(segments, transcriptions, srt_path)


# Пример использования
video_path = "/content/Heretic.2024.SAMPLE.mkv"
audio_path = "/content/extracted_audio.wav"
srt_path = "/content/subtitles.srt"

main(video_path, audio_path, srt_path)


Аудио успешно извлечено: /content/extracted_audio.wav
Обнаружено 86 сегментов речи.
Модель jonatasgrosman/wav2vec2-large-xlsr-53-russian загружена.
Распознавание сегмента 1/86 (1.16%)...
Распознавание сегмента 2/86 (2.33%)...
Распознавание сегмента 3/86 (3.49%)...
Распознавание сегмента 4/86 (4.65%)...
Распознавание сегмента 5/86 (5.81%)...
Распознавание сегмента 6/86 (6.98%)...
Распознавание сегмента 7/86 (8.14%)...
Распознавание сегмента 8/86 (9.30%)...
Распознавание сегмента 9/86 (10.47%)...
Распознавание сегмента 10/86 (11.63%)...
Распознавание сегмента 11/86 (12.79%)...
Распознавание сегмента 12/86 (13.95%)...
Распознавание сегмента 13/86 (15.12%)...
Распознавание сегмента 14/86 (16.28%)...
Распознавание сегмента 15/86 (17.44%)...
Распознавание сегмента 16/86 (18.60%)...
Распознавание сегмента 17/86 (19.77%)...
Распознавание сегмента 18/86 (20.93%)...
Распознавание сегмента 19/86 (22.09%)...
Распознавание сегмента 20/86 (23.26%)...
Распознавание сегмента 21/86 (24.42%)...
Распозна